# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
print(link_system_prompt)

In [6]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [ ]:
print(get_links_user_prompt(ed))

In [7]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [8]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/moonshotai/Kimi-K2-Instruct',
 '/mistralai/Voxtral-Mini-3B-2507',
 '/mistralai/Voxtral-Small-24B-2507',
 '/Chain-GPT/Solidity-LLM',
 '/LGAI-EXAONE/EXAONE-4.0-32B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/llamameta/Grok-4-heavy-free',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/akhaliq/anycoder',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces',
 '/datasets/NousResearch/Hermes-3-Dataset',
 '/datasets/common-pile/caselaw_access_project',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/microsoft/rStar-Coder',
 '/datasets/snorkelai/agent-finance-reasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/gram

In [9]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [10]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://huggingface.co"))

In [11]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [12]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [13]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2-Instruct\nUpdated\n3 days ago\n•\n145k\n•\n1.56k\nmistralai/Voxtral-Mini-3B-2507\nUpdated\n2 days ago\n•\n14.3k\n•\n352\nmistralai/Voxtral-Small-24B-2507\nUpdated\n2 days ago\n•\n552\n•\n300\nChain-GPT/Solidity-LLM\nUpdated\n27 days ago\n•\n31\n•\n292\nLGAI-EXAONE/EXAONE-4.0-32B\nUpdated\n2 days ago\n•\n272k\n•\n175\nBrowse 1M+ models\nSpaces\nRunning\n10.5k\n10.5k\nDeep

In [14]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [15]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Brochure

## About Us
**Hugging Face** is at the forefront of building the future of AI. As a leading community-driven platform, we enable collaboration among machine learning enthusiasts, researchers, and engineers. Our mission is to make machine learning accessible and collaborative, offering state-of-the-art tools to accelerate the development of AI solutions.

## Our Platform
- **Models**: Explore and collaborate on over **1 million ML models** tailored for a variety of applications.
- **Datasets**: Access and share more than **250,000 datasets**, helping researchers find the right data for their projects.
- **Spaces**: Discover and deploy applications with ease, hosting tailored solutions for different AI needs.
- **Community**: Engage with a vibrant community of over **50,000 organizations**, sharing knowledge and expertise.

## Our Solutions
### For Businesses
We offer **Paid Compute** and **Enterprise Solutions** designed for organizations of all sizes. For this, we provide:
- Optimized inference endpoints for deploying models.
- Advanced security features and dedicated support for team environments.
- Enterprise-grade tools starting at **$20/user/month**.

### For Developers
Join the movement in open-source AI tools! Our repository includes:
- **Transformers**: State-of-the-art models compatible with PyTorch and TensorFlow.
- **Diffusers**: Advanced tools for diffusion model deployment.
- **Tokenizers**: Fast tokenizers optimized for both research and production.

## Company Culture
At Hugging Face, we foster a collaborative and inclusive culture, encouraging our team members to innovate and learn continuously. We believe that the future of AI is a community effort, and we actively promote openness and transparency in our operations. Our culture is built on teamwork, where individuals feel valued and empowered to contribute.

## Careers
Join the Hugging Face family! We are constantly looking for passionate individuals who want to make a difference in the AI landscape. Whether you’re a developer, researcher, or business professional, if you share our vision of building a better future with AI, we want to hear from you.

**Current Openings**: Visit our [Jobs Page](https://huggingface.co/jobs) for more information about opportunities to contribute to our mission.

## Join Us
Let’s build the future together! Explore our [website](https://huggingface.co) to learn more about our offerings, community initiatives, and how you can get involved. 

**Connect with us on Social Media**:  
- **Twitter**: [@huggingface](https://twitter.com/huggingface)  
- **LinkedIn**: [Hugging Face](https://www.linkedin.com/company/huggingface)  
- **GitHub**: [Hugging Face Repository](https://github.com/huggingface)

Together, we can turn ideas into reality and harness the power of AI for everyone.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [16]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [18]:
stream_brochure("Il-Cubo Corsi e Ripetizioni", "https://il-cubo.it/")

Found links: {'links': [{'type': 'about page', 'url': 'https://il-cubo.it/chi-siamo'}, {'type': 'services page', 'url': 'https://il-cubo.it/cosa-offriamo'}, {'type': 'careers page', 'url': 'http://il-cubo.it/collabora/offerte-di-lavoro'}, {'type': 'collaboration page', 'url': 'http://il-cubo.it/collabora'}, {'type': 'events page', 'url': 'https://soci.il-cubo.it/eventi/'}]}


# Il-Cubo Corsi e Ripetizioni Brochure

## About Us
Il-Cubo is a dynamic cooperative founded in 2009, originating from a group of university students who recognized the need for effective communication in education. Our mission is to change the way knowledge is shared, emphasizing encouragement and personalized methods tailored to each student's unique abilities and motivations. We are committed to finding the *good* in every student and helping them harness their potential.

## Our Unique Method
At Il-Cubo, we believe that every student has the capability to excel. Our approach is rooted in motivation and personalized instruction, adapting to the individual needs of each learner. Whether it’s mastering complex subjects or enhancing study techniques, our mentors are dedicated to providing effective, innovative, and engaging tutoring experiences.

## Services Offered
We cater to students of all ages and backgrounds, providing tutoring and courses across a wide range of subjects, including but not limited to:
- Mathematics
- Science (Physics, Chemistry)
- Languages (Latin, Spanish, Greek)
- Economy
- Computer skills
- Study skills workshops

## Join Our Team
We are always looking to expand our network of passionate tutors. If you're interested in becoming part of our educational community, we invite you to collaborate with us as a tutor. We value your expertise and innovative teaching methodologies that can spark a positive learning experience for students.

### Career Opportunities
Il-Cubo offers various job positions for enthusiastic individuals who want to make a difference in education. We prioritize potential candidates who are not only knowledgeable but also possess the ability to communicate effectively with students.

## Our Philosophy
Education should not just be about lectures; it should be about communication and understanding. Our diverse team of students and professionals is dedicated to finding the best solutions to foster learning and personal growth among students. We strive to create an environment that inspires learning and enhances student confidence.

## Connect with Us
Stay updated on our latest programs, events, and job offers by following us on our social media channels. We also encourage interested individuals to subscribe to our newsletter for news and opportunities.

### Contact Us
For more information or inquiries, please feel free to reach out:
- Phone: 340 1944787 / 02 9954052
- Email: [info@ilcubo.it](mailto:info@ilcubo.it)

Join us at Il-Cubo and help enrich the educational journey of our students!

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>